<a href="https://colab.research.google.com/github/Coralain10/ML_Archaeological-Objects-Reconstruction/blob/main/fracturing_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import math
import numpy as np

## Fracturar

In [2]:
def get_fractured(source, min_points=1, max_points=4, min_radius=3, max_radius=6, sphere_chance=0.75):
    fractured = source.copy()
    points = np.random.randint(min_points, max_points+1)
    idxs = np.argwhere(fractured == 1)
    centers = idxs[np.random.choice(len(idxs), points, False)]
    
    for x,y,z in centers:
        r = np.random.randint(min_radius, max_radius+1)
        xmin, xmax = max(0, x-r), x+r
        ymin, ymax = max(0, y-r), y+r
        zmin, zmax = max(0, z-r), z+r
        sphere = np.ones_like(fractured)
        sphere[xmin:xmax, ymin:ymax, zmin:zmax] = 0
        # sphere or cube
        if np.random.rand() < sphere_chance:
            idxs = np.argwhere(sphere == 0)
            idxs_remove = np.sqrt((idxs[:,0] - x)**2 + (idxs[:,1] - y)**2 + (idxs[:,2] - z)**2)
            idxs_remove = idxs[idxs_remove > r]
            sphere[idxs_remove[:,0], idxs_remove[:,1], idxs_remove[:,2]] = 1
            
        fractured *= sphere
    
    return fractured

## Gráfico

In [3]:
def read_points(file):
	with open(file) as f:
		d = [l.strip().split(' ') for l in f.readlines()]
		d = [[float(n) for n in l[1:]] for l in d if l[0] == 'v']
		d = np.asarray(d, dtype=np.float32)
	
	return d

def read_point_files(files):
	out = []
	for file in files:
		out.append(read_points(file))
	
	return out

def point_cloud_to_volume(points, vsize, radius=1.0):
    """ input is Nx3 points.
        output is vsize*vsize*vsize
        assumes points are in range [-radius, radius]
    """
    vol = np.zeros((vsize,vsize,vsize), dtype=np.bool)
    voxel = 2*radius/float(vsize)
    locations = (points + radius)/voxel
    locations = locations.astype(int)
    vol[locations[:,0],locations[:,1],locations[:,2]] = 1.0
    
    return vol

def volume_to_point_cloud(vol):
    """ vol is occupancy grid (value = 0 or 1) of size vsize*vsize*vsize
        return Nx3 numpy array.
    """
    vsize = vol.shape[0]
    assert(vol.shape[1] == vsize and vol.shape[1] == vsize)
    points = []
    for a in range(vsize):
        for b in range(vsize):
            for c in range(vsize):
                if vol[a,b,c] == 1:
                    points.append(np.array([a,b,c]))
    if len(points) == 0:
        return np.zeros((0,3))
    points = np.vstack(points)
    
    return points

def auto_pcl_to_volume(points, vsize):
	data_min = np.min(points)
	data_max = np.max(points)
	radius = max(abs(data_min), data_max)
	radius = math.ceil(radius*100) / 100
	vol = point_cloud_to_volume(points, vsize, radius)

	return vol

# Implementación de la data

In [71]:
!wget https://github.com/Coralain10/ML_Archaeological-Objects-Reconstruction/blob/main/data/Full.zip

--2022-10-24 04:01:28--  https://github.com/Coralain10/ML_Archaeological-Objects-Reconstruction/blob/main/data/Full.zip
Resolving github.com (github.com)... 192.30.255.112
Connecting to github.com (github.com)|192.30.255.112|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘Full.zip’

Full.zip                [ <=>                ] 137.76K  --.-KB/s    in 0.02s   

2022-10-24 04:01:29 (6.09 MB/s) - ‘Full.zip’ saved [141070]



In [72]:
from zipfile import ZipFile, ZIP_DEFLATED

In [73]:
zip_name = "/content/Full.zip"

In [75]:
with ZipFile(zip_name, 'r') as zip:
  zip.printdir()
  zip.extractall()

File Name                                             Modified             Size
Full.npy                                       2022-10-24 03:06:12     29184594


In [76]:
filedir = "/content/Full.npy"

In [77]:
data = np.load(filedir, allow_pickle=True)

In [78]:
nrepets = 8
dc = np.array([ data.copy() for _ in range(nrepets) ])
print(dc.shape)

(8, 889)


In [79]:
ds = []
for i in range(nrepets):
  for j in range(dc.shape[1]):
    ds += [ dc[i][j] ]
ds = np.array(ds)
ds.shape

(7112, 32, 32, 32)

In [80]:
fc = []
for f in ds:
  fc += [get_fractured(f)]
fc = np.array(fc)

In [81]:
dataset = np.array([ ds, fc ]) #completo | fracturado

In [82]:
file_name = 'FullAndFracture.npy'
np.save(file_name,dataset)

In [83]:
from zipfile import ZipFile, ZIP_DEFLATED

In [84]:
with ZipFile('FullAndFracture.zip','w') as zip: zip.write(file_name, compress_type=ZIP_DEFLATED)